In [1]:
import sys
sys.path.append('..')
from symphony import datatypes
from mp_api.client import MPRester
import numpy as np
import jraph
import matscipy
import matscipy.neighbours
import jax.numpy as jnp

/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [5]:
# query and download data
with MPRester("NA4RS6zGonPp3S3TQTSBPkzevjE3jAPt") as mpr:
    docs = mpr.summary.search(elements=["Si", "O"], num_elements=(2, 2))

/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/mp_api/client/mprester.py:182: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Retrieving SummaryDoc documents:   0%|          | 0/344 [00:00<?, ?it/s]

In [4]:
docs[0]

MPDataDoc<SummaryDoc>(
builder_meta=EmmetMeta(emmet_version='0.38.6', pymatgen_version='2022.10.22', pull_request=825, database_version='2022.10.28', build_date=datetime.datetime(2022, 10, 28, 0, 53, 42, 312000)),
nsites=45,
elements=[Element Bi, Element O, Element Sb, Element Si],
nelements=4,
composition=Comp: Si2 Bi13 Sb2 O28,
composition_reduced=Comp: Si2 Bi13 Sb2 O28,
formula_pretty='Si2Bi13(SbO14)2',
formula_anonymous='A2B2C13D28',
chemsys='Bi-O-Sb-Si',
volume=763.6171111879416,
density=7.533623425275825,
density_atomic=16.969269137509816,
symmetry=SymmetryData(crystal_system=<CrystalSystem.tri: 'Triclinic'>, symbol='P-1', number=2, point_group='-1', symprec=0.1, version='1.16.2'),
property_name='summary',
material_id=MPID(mp-1044098),
deprecated=False,
deprecation_reasons=None,
last_updated=datetime.datetime(2022, 10, 28, 0, 53, 42, 312000),
origins=[],
warnings=[],
structure=Structure Summary
Lattice
    abc : 6.126400651691089 10.212474297787534 12.694139193021677
 angles : 86

In [41]:
type(structure), type(structure.structure)

(pydantic.main.MPDataDoc, pymatgen.core.structure.Structure)

In [28]:
structure.structure.lattice.matrix

array([[ 5.968034, -0.067694,  1.382307],
       [ 1.761293, 10.046719,  0.505881],
       [-0.075195,  0.07781 , 12.693678]])

In [21]:
structure.structure.cart_coords

array([[ 2.20389731,  3.36142822, 13.14942728],
       [ 5.47764055,  6.69893302,  1.43743833],
       [ 1.31126542,  6.48732282,  7.60038907],
       [ 4.70239668,  3.5102862 ,  3.11791844],
       [ 2.63651446,  3.75853204,  9.03168455],
       [ 6.32459547,  3.52989359,  6.99648234],
       [ 2.9702088 ,  6.5477355 , 11.46441413],
       [ 3.34973637,  9.40183839,  6.52562967],
       [ 2.37955245,  6.44142285,  2.28286788],
       [ 1.69033505, 10.12409125, 13.19935453],
       [ 2.44369688,  0.0285659 ,  2.51536671],
       [ 5.28250204,  3.61438384, 12.31708051],
       [ 5.01390556,  6.27161798,  5.5656868 ],
       [ 5.21718937, 10.02991972, 12.075694  ],
       [ 4.28683119,  0.61134318,  8.05471568],
       [ 6.80790247,  9.99593611,  5.32876399],
       [ 2.60448673, 10.08951745,  9.75898045],
       [ 4.84926877,  4.08613588,  5.175869  ],
       [ 1.99566866,  4.49082591,  7.0815731 ],
       [ 0.6285265 ,  3.27858447,  0.30792586],
       [ 2.83560729,  1.83732156, 13.230

In [38]:
# def ase_atoms_to_jraph_graph(
#     atoms: ase.Atoms, atomic_numbers: jnp.ndarray, nn_cutoff: float
# ) -> jraph.GraphsTuple:
# Create edges
receivers, senders = matscipy.neighbours.neighbour_list(
    quantities="ij", positions=structure.structure.cart_coords, cutoff=nn_cutoff, cell=np.eye(3)
)  # what is this cell actually, it isn't always going to be eye(3) is it


In [40]:
# Get the species indices
species = np.searchsorted([8, 14], structure.structure.atomic_numbers)

jraph.GraphsTuple(
    nodes=datatypes.NodesInfo(np.asarray(structure.structure.cart_coords), np.asarray(species)),
    edges=np.ones(len(senders)),
    globals=None,
    senders=np.asarray(senders),
    receivers=np.asarray(receivers),
    n_node=np.array([len(structure.structure.cart_coords)]),
    n_edge=np.array([len(senders)]),
)

GraphsTuple(nodes=NodesInfo(positions=array([[ 2.20389731,  3.36142822, 13.14942728],
       [ 5.47764055,  6.69893302,  1.43743833],
       [ 1.31126542,  6.48732282,  7.60038907],
       [ 4.70239668,  3.5102862 ,  3.11791844],
       [ 2.63651446,  3.75853204,  9.03168455],
       [ 6.32459547,  3.52989359,  6.99648234],
       [ 2.9702088 ,  6.5477355 , 11.46441413],
       [ 3.34973637,  9.40183839,  6.52562967],
       [ 2.37955245,  6.44142285,  2.28286788],
       [ 1.69033505, 10.12409125, 13.19935453],
       [ 2.44369688,  0.0285659 ,  2.51536671],
       [ 5.28250204,  3.61438384, 12.31708051],
       [ 5.01390556,  6.27161798,  5.5656868 ],
       [ 5.21718937, 10.02991972, 12.075694  ],
       [ 4.28683119,  0.61134318,  8.05471568],
       [ 6.80790247,  9.99593611,  5.32876399],
       [ 2.60448673, 10.08951745,  9.75898045],
       [ 4.84926877,  4.08613588,  5.175869  ],
       [ 1.99566866,  4.49082591,  7.0815731 ],
       [ 0.6285265 ,  3.27858447,  0.30792586],
  